<img align="right" width="200" height="200" src="https://avatars3.githubusercontent.com/u/43672704?s=400&u=7f10d18e6375065a2bd501c9cfd59a2ac6ad0f80&v=4">

# MAPD B final project - Data Management

**Authors:**
* Alessandro Lambertini - ID: 1242885
* Michele Guadagnini    - ID: 1230663


# Exercise 1 - Redundancy

We are programming a file based RAID-4 software algorithm. For this purpose we are converting a single input (`raid4.input`) file into four data files `raid4.0, raid4.1, raid4.2, raid4.3` and one parity file `raid4.4` - the four data and one parity file are called *stripe files*. <br>

To do this we are reading in a loop sequentially blocks of four bytes from the input file until the whole file is read:
* in each loop we write one of the four read bytes round-robin to each data file, compute the parity of the four input bytes and write the result into the fifth parity file.
* we continue until all input data has been read. If the last bytes read from the input file are not filling four bytes, we consider the missing bytes as zero for the parity computation.


In [1]:
# import needed libraries
import os

## Full exercise code

Here we report the full code of the whole exercise. All the assignment points from **1.1** to **1.5** will be answered below.

In [2]:
filename = "raid4.input"
chunk_size = 4

cols_parity = [0,0,0,0,0] # list for column-wise parities
SumP6 = 0                 # sum of parity of the five stripe files (6th-parity)

# creating the stripe files
raid40 = open("raid4.0", "wb")
raid41 = open("raid4.1", "wb")
raid42 = open("raid4.2", "wb")
raid43 = open("raid4.3", "wb")
raid44 = open("raid4.4", "wb")

# loading input file
with open(filename, "rb") as fin:
    AllBytes = fin.read()
    
for i in range(0,len(AllBytes),chunk_size):
    chunk = AllBytes[i:i+chunk_size]
    
    if (len(chunk)<4):
        chunk += (0).to_bytes(1,'big') # padding
    
    # filling stripe files
    raid40.write(chunk[0].to_bytes(1, byteorder='big'))
    raid41.write(chunk[1].to_bytes(1, byteorder='big'))
    raid42.write(chunk[2].to_bytes(1, byteorder='big'))
    raid43.write(chunk[3].to_bytes(1, byteorder='big'))
     
    # row-wise parity stripe file
    parity = 0
    for j in chunk:
        parity ^= j
    raid44.write(parity.to_bytes(1, byteorder='big'))
    
    # column-wise parity calculation
    for k in range(len(chunk)):
        cols_parity[k] ^= chunk[k]
    cols_parity[4] ^= parity
    
    # 6th-parity calculation
    parity6th = 0
    for j in chunk:
        parity6th ^= j
    parity6th ^= parity
    
    # updating the sum of 6th-parity
    SumP6 += parity6th
    
raid40.close()
raid41.close()
raid42.close()
raid43.close()
raid44.close()


## 1.1) Produce the stripe files

The program has created the five stripe files. Here is a check: the first column represent the size of the file.

In [3]:
!ls -sh | grep "raid4\."

 44K raid4.0
 44K raid4.1
 44K raid4.2
 44K raid4.3
 44K raid4.4
168K raid4.input


## 1.2) Column-wise parities and size overhead

Column-wise parity acts as a *checksum* for each stripe file. Indeed, *checksum* consists in summing up all the bits and storing it somewhere. In this way, to check the file integrity after a transmission it is possible to do again the *checksum* and compare its result with the previous one. If it is different it means that some corruption occurred during transmission.

In [4]:
# column-wise parities
print("Column-wise parities: ", [hex(p) for p in cols_parity])

# checking the size overhead
InputSize = os.stat(filename).st_size

AllStripeFiles = 0
for i in range(5):
    name = "raid4."+str(i)
    AllStripeFiles += os.stat(name).st_size
    
print("The size overhead is: {:.2f} %".format((AllStripeFiles-InputSize)/InputSize * 100))

Column-wise parities:  ['0xa5', '0x7', '0xa0', '0x9b', '0x99']
The size overhead is: 25.00 %


## 1.3) 5-byte parity in hex format

We just need to stick together the column-wise parities obtained above, taking care of padding with 0 when needed.

In [5]:
# 5-byte parity
P5 = "0x"+''.join('%02x'%i for i in cols_parity)
print("5-byte parity is: ", P5)

5-byte parity is:  0xa507a09b99


## 1.4) 6th stripe file

The content of a 6th stripe file containing the row-wise parity of the five stripe files would be all zeros. <br>
Indeed, naming a byte of each of the four stripe files as *X,Y,Z,T* and a byte of the parity file as *P* : <br>
`6thParity = X^Y^Z^T^P` , but since `P = X^Y^Z^T` , we have that `6thParity = P^P = 0`

To check this, we have computed the sum of all the bytes of this 6th stripe file.

In [6]:
print("The sum of all the parity bits in the 6th stripe file is: ", SumP6)

The sum of all the parity bits in the 6th stripe file is:  0


## 1.5) File reconstruction

Comparing the 5-byte obtained before ( `0xa507a09b99` ) with the one provided by the exercise ( `0xff07a09b99` ) we can see that the corrupted stripe file is the first one, since it is the only one with a different column-wise parity.

So we can use all the other stripe files and also the parity file to reconstruct it. Naming *X, Y, Z, T* the stripe bytes and *P* the parity one, we can reconstruct *X* by doing: `X = P^Y^Z^T`. Indeed: <br>
`P^Y^Z^T = (X^Y^Z^T)^Y^Z^T = X^(Y^Y)^(Z^Z)^(T^T) = X^0^0^0 = X`

Once reconstructed the first stripe file, we can rebuild the original file by putting together the stripe files following the correct order. 

Since we padded the last stripe file with an additional 0 byte to have all the stripes with the same lenght, the reconstructed input file is not exactly equal without removing this last byte. So, for perfect reconstruction, it would be useful to store somewhere also the original size of the input file.

In [7]:
# reconstruction of first stripe file raid4.0
raid41 = open("raid4.1", "rb")
raid42 = open("raid4.2", "rb")
raid43 = open("raid4.3", "rb")
raid44 = open("raid4.4", "rb")

# loading the stripe files
stripe1 = raid41.read()
stripe2 = raid42.read()
stripe3 = raid43.read()
parity  = raid44.read()

raid41.close()
raid42.close()
raid43.close()
raid44.close()

# reconstruction of first stripe
stripe0_reconstructed = []
for (y,z,t,p) in zip(stripe1,stripe2,stripe3,parity):
    stripe0_reconstructed.append( y^z^t^p )
    
# saving the reconstructed first stripe
with open("raid4_reconstructed.0","wb+") as outfile:
    for ii in bytes(stripe0_reconstructed):
        outfile.write(ii.to_bytes(1, byteorder='big'))
        
# reconstruction of original input file
input_reconstructed = []
for (x,y,z,t) in zip(stripe0_reconstructed,stripe1,stripe2,stripe3):
    input_reconstructed.append(x)
    input_reconstructed.append(y)
    input_reconstructed.append(z)
    input_reconstructed.append(t)
    
# saving the reconstructed input file
with open("raid4_reconstructed.input","wb+") as outfile:
    for ii in bytes(input_reconstructed):  
        outfile.write(ii.to_bytes(1, byteorder='big'))
        

In [8]:
# check difference between original and reconstructed first stripe
!diff -s raid4_reconstructed.0 raid4.0

Files raid4_reconstructed.0 and raid4.0 are identical


In order to perfectly reconstruct the original file, we needed to remove the last byte coming from the last stripe, since it comes from padding.

In [9]:
# check difference between original and reconstructed input file
!diff -s raid4_reconstructed.input raid4.input

Binary files raid4_reconstructed.input and raid4.input differ


In [10]:
# saving the reconstructed input file without last byte
with open("raid4_reconstructed.input","wb+") as outfile:
    # below we added [:-1] to remove the last byte used for padding the last stripe
    for ii in bytes(input_reconstructed[:-1]):  
        outfile.write(ii.to_bytes(1, byteorder='big'))
        
# check difference between original and reconstructed input file
!diff -s raid4_reconstructed.input raid4.input

Files raid4_reconstructed.input and raid4.input are identical
